In [62]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
sns.set()
pd.options.display.max_rows = 100

df = pd.read_csv('CR_FC_PREP_0522.csv')

df.head()
df.info()
print(f'Dataset size: {df.shape}')

,POD,RATE
0,2005-07-19,2200.0
1,2005-07-20,2250.0
2,2005-07-21,2300.0
3,2005-07-22,2280.0
4,2005-07-23,2260.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6375 entries, 0 to 6374
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   POD     6375 non-null   object 
 1   RATE    6375 non-null   float64
dtypes: float64(1), object(1)
memory usage: 99.7+ KB
Dataset size: (6375, 2)


In [63]:
df['POD'] = pd.to_datetime(df['POD'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6375 entries, 0 to 6374
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   POD     6375 non-null   datetime64[ns]
 1   RATE    6375 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 99.7 KB


In [64]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Y is the target
Y = df.pop('RATE') 
X = df

# Normalise and split the data into training and testing sets
sc = StandardScaler()
X = sc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

<h3>Tunnning</h3>

In [65]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

def train_evaluate_DTR(X_train, X_test, y_train, y_test):
    # Define the decision tree regressor
    dt_regressor = DecisionTreeRegressor()

    # Define the hyperparameters to search over
    param_grid = {
        'max_depth': [3, 5, 7, 10, 15],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
    }

    # Perform a grid search over the hyperparameters using cross-validation
    grid_search = GridSearchCV(dt_regressor, param_grid, cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Print the best hyperparameters found by the grid search
    print('Best hyperparameters:', grid_search.best_params_)

    # Calculate the test accuracy of the model with the best hyperparameters
    best_model = grid_search.best_estimator_
    best_model.fit(X_train, y_train)
    test_acc = best_model.score(X_test, y_test)

    # Print the test accuracy
    print('Test accuracy:', test_acc)

    return best_model

In [66]:
best_model = train_evaluate_DTR(X_train, X_test, y_train, y_test)

Best hyperparameters: {'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 2}
Test accuracy: 0.9958132447314251


In [69]:
from sklearn.metrics import mean_squared_error, r2_score

def findDT(best_model):
    # Train the model
    model = best_model
    model.fit(X_train, y_train)

    # Make predictions on test data
    predictions = model.predict(X_test)

    # Calculate mean squared error between actual and predicted values
    mse = mean_squared_error(y_test, predictions)

    # Calculate root mean squared error
    rmse = np.sqrt(mse)

    # Calculate R-squared value
    r2 = r2_score(y_test, predictions)

    # Calculate mean value of your actual data
    mean = np.mean(y_test)

    # Calculate accuracy of your model in percentage
    accuracy = (1 - rmse/mean) * 100

    print('Root Mean Squared Error:', round(rmse, 3))
    print('R-squared:', round(r2, 3))
    print(f"Accuracy: {accuracy:.2f}%")

In [68]:
findDT(best_model)

Root Mean Squared Error: 128.71
R-squared: 0.996
Accuracy: 91.96%


Similar to hypertunning, the default parameter may have been chosen based on their good performance on a specific dataset or task, and so deviating too much from those defaults may not be beneficial in all cases. Hence, further tuning may not be necessary or useful.